#### Exploring Which Dataset to Use

In [1]:
import pandas as pd
import os
import numpy as np
from difflib import get_close_matches

os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/woba modeling/data/')

#### Load Data

In [2]:
m1_woba = pd.read_csv('sim_results/m1_wobacon_23_25.csv')
m2_woba = pd.read_csv('sim_results/m2_wobacon_23_25.csv')
ev_dir = pd.read_csv('sim_results/ev_wobacon_23_25.csv')
full = pd.read_csv('transforms/wobacon.csv')
woba = pd.read_csv('pitch/savant.csv')

#### Correlations

In [34]:
merged = woba.merge(full[['name', 'year', 'count', 'monte']], on=['name', 'year'], how='right')
# merged = merged.merge(ev_dir[['name', 'year', 'monte']], on = ['name', 'year'], how='right', suffixes=('', '_ev'))
# merged = merged.merge(m2_woba[['name', 'year', 'count', 'monte']], on=['name', 'year'], how='right', suffixes=('', '_m2'))
# merged = merged.rename(columns={'count': 'count_m1', 'monte': 'm1_monte', 'count_m2': 'count_m2', 'monte_m2': 'm2_monte'})

In [35]:
missing_names = merged[merged.isnull().any(axis=1)]['name'].unique()
existing_names = merged[~merged.isnull().any(axis=1)]['name'].unique()

name_map = {}
for name in missing_names:
    matches = get_close_matches(name, existing_names, n=1, cutoff=0.8)
    if matches:
        name_map[name] = matches[0]



In [36]:
for missing_name, matched_name in name_map.items():
    idx_missing = merged[(merged['name'] == missing_name) & (merged.isnull().any(axis=1))].index
    for idx in idx_missing:
        year = merged.loc[idx, 'year']
        match_row = merged[(merged['name'] == matched_name) & (merged['year'] == year) & (~merged.isnull().any(axis=1))]
        if not match_row.empty:
            for col in merged.columns:
                if pd.isnull(merged.at[idx, col]):
                    merged.at[idx, col] = match_row.iloc[0][col]

In [37]:
merged = merged[['name', 'year', 'monte', 'wobacon', 'xwobacon', 'pa', 'count']]

In [38]:
merged = merged[merged['count'] > 60]

#### Correlation

stability

In [39]:
stats = ['monte', 'wobacon', 'xwobacon']
corr_next = merged.copy()

for stat in stats:
    corr_next[f'{stat}_next'] = corr_next.groupby('name')[stat].shift(-1)

corr_next = corr_next.dropna(subset=[f'{stat}_next' for stat in stats])

corrs = {}
for stat in stats:
    corrs[stat] = corr_next[[stat, f'{stat}_next']].corr().iloc[0, 1]
    corrs[stat] = corrs[stat] ** 2

print(corrs)

{'monte': 0.558718178723602, 'wobacon': 0.14716458742065097, 'xwobacon': 0.4201046749808671}


dataset to next year wobacon

In [40]:
prev_stat_next_woba_corr = {}
for stat in stats:
    prev_stat_next_woba_corr[stat] = corr_next[[stat, 'wobacon_next']].corr().iloc[0, 1]
    prev_stat_next_woba_corr[stat] = prev_stat_next_woba_corr[stat] ** 2

print(prev_stat_next_woba_corr)

{'monte': 0.2819980904367422, 'wobacon': 0.14716458742065097, 'xwobacon': 0.2531463954231565}


2023 stat to 2024 wobacon

In [41]:
stat_2024_to_2025_woba_corr = {}
df_2024 = corr_next[corr_next['year'] == 2023]
for stat in stats:
    stat_2024_to_2025_woba_corr[stat] = df_2024[[stat, 'wobacon_next']].corr().iloc[0, 1]
    stat_2024_to_2025_woba_corr[stat] = stat_2024_to_2025_woba_corr[stat] ** 2

print(stat_2024_to_2025_woba_corr)

{'monte': 0.28175017271054587, 'wobacon': 0.1396256367051462, 'xwobacon': 0.260914572190645}


2024 stat to 2025 wobacon

In [42]:
stat_2024_to_2025_woba_corr = {}
df_2024 = corr_next[corr_next['year'] == 2024]
for stat in stats:
    stat_2024_to_2025_woba_corr[stat] = df_2024[[stat, 'wobacon_next']].corr().iloc[0, 1]
    stat_2024_to_2025_woba_corr[stat] = stat_2024_to_2025_woba_corr[stat] ** 2

print(stat_2024_to_2025_woba_corr)

{'monte': 0.28845990185641834, 'wobacon': 0.16529401828753343, 'xwobacon': 0.25630883183791486}


xwobacon to monte

In [43]:
print(merged['monte'].corr(merged['xwobacon']))
print(merged['monte'].corr(merged['wobacon']))
print(merged['wobacon'].corr(merged['xwobacon']))


0.901786239367908
0.7549035825042856
0.8191918149251566


#### Diffrences

In [45]:
merged['diff'] = abs(merged['monte'] - merged['xwobacon'])
print(merged['diff'].nlargest(10))

906     0.163007
1695    0.098439
250     0.097923
876     0.088361
747     0.087013
1357    0.084943
1632    0.083621
1944    0.083390
1663    0.083389
940     0.082204
Name: diff, dtype: float64
